# Portfolio Optimisation

Portfolio optimization is the process of choosing the best combination of assets to achieve the highest possible return for a given level of risk, or alternatively, the lowest risk for a desired return.  The goal is to build a well-diversified portfolio that performs efficiently under various market conditions.

## Modern Portfolio Theory


Modern Portfolio Theory (MPT), also known as Mean-Variance Portfolio Theory, represents a major breakthrough in finance. It is based on the premise that asset returns are normally distributed, meaning that their behavior can be described using just the mean (expected return) and variance (risk or volatility).

The core idea of MPT is to achieve diversification by constructing a portfolio that either minimizes risk for a given level of expected return or maximizes expected return for a given level of risk.

The Efficient Frontier represents the set of optimal portfolios along the risk-return spectrum. Portfolios that lie below the Efficient Frontier are considered sub-optimal, as they offer lower returns for a given level of risk or higher risk for a given level of return.

**Portfolios on the Efficient Frontier provide:**

a. The highest expected return for a given level of risk`, or

b. The lowest level of risk for a given level of expected return`

In essence, an investor's goal is to determine the level of risk they are comfortable with, and then select a portfolio on the Efficient Frontier that offers the best possible return for that risk level. 

**Import Libraries**

In [ ]:
# Import libraries
import sys
import os
sys.path.append(os.path.abspath(".."))

import numpy as np
import pandas as pd
import cvxpy as cp

import quantmod.charts
import plotly.graph_objects as go

from utils import query_all_stocks

## Data Retrieval

We will retrieve price data for selected stocks from our database to build our portfolio.

In [ ]:
# Query stock data for all 49 stocks
df = query_all_stocks()
assets = sorted(['ICICIBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT'])
df = df[assets]
df.head()

In [ ]:
# Visualise stock data
df.iplot(kind='normalized', showlegend=True)

In [ ]:
# Dataframe of returns and volatility
returns = df.pct_change().dropna()
annual_returns = round(returns.mean()*260*100,2)
annual_stdev = round(returns.std()*np.sqrt(260)*100,2)

# Subsume into dataframe
stats = pd.DataFrame({
    'Ann Ret': annual_returns,
    'Ann Vol': annual_stdev
})

# Get the output
stats.head()

In [ ]:
# Plot annualized return and volatility
stats.iplot(kind='bar', title='Annualized Return & Volatility (%)', showlegend=True)

**Portfolio Composition before Optimization**

In [ ]:
# Update quantmod to latest version
# Portfolio composition
stats['Ann Vol'].iplot(kind='pie', showlegend=True, title='Annualised Volatility')

## Portfolio Statistics

Consider a portfolio which is fully invested in risky assets. Let $w$ and $\mu$ be the vector of weights and mean returns of *n* assets. <br><br>

$$
\ {w=}\left( 
\begin{array}{c}
w_1 \\
w_2 \\
\vdots \\
w_n \\ 
\end{array}%
\right);
\ \mathbf{\mu=}\left( 
\begin{array}{ccc}
\mu_1 \\ 
\mu_2 \\ 
\vdots \\
\mu_n \\ 
\end{array}%
\right)
$$ 

where the $\sum_{i=1}^{n}w_i=1$

**`Expected Portfolio Return`** is then the dot product of the expected returns and their weights. <br><br>

$$\mu_\pi = w^T\cdot\mu$$

which is also equivalent to the $\Sigma_{i=1}^{n}w_i\mu_i$


**`Expected Portfolio Variance`** is then the multidot product of weights and the covariance matrix. <br><br>

$$\sigma^2_\pi = w^T\cdot\Sigma\cdot w $$

where, ${\Sigma}$ is the covariance matrix

$$
{\Sigma=}\left( 
\begin{array}{ccc}
\Sigma_{1,1} & \dots & \Sigma_{1,n} \\ 
\vdots & \ddots & \vdots  \\ 
\Sigma_{n,1} & \dots & \Sigma_{n,n} \\ %
\end{array}%
\right)
$$

In [ ]:
# Compute statistics
mean_returns = (returns.mean()*260).values
cov_matrix = (returns.cov() * 260).values
n = len(mean_returns) 

**Maximum Sharpe Ratio Portfolio**

CVXPY is designed specifically for convex optimization, while the original Sharpe ratio maximization problem is non-convex due to its fractional (ratio) form.

To address this, we apply a mathematical transformation. We set the portfolio excess return equal to 1 and then find the portfolio that achieves exactly 1 unit of excess return with minimum risk. This works due to the homogeneity of the Sharpe ratio. If a set of portfolio weights $w^{*}$ maximizes the Sharpe ratio, then any scaled version $k * w^{*}$ also achieves the same Sharpe ratio.

In [ ]:
# ---- 1. Maximum Sharpe Ratio Portfolio ----

def optimize_max_sharpe(mean_returns, cov_matrix, risk_free_rate=0.0):
    
    w = cp.Variable(n)
    excess_return = mean_returns - risk_free_rate
    port_return =  excess_return @ w
    port_risk = cp.quad_form(w, cov_matrix)
    
    # Minimize risk for 1 unit of excess return
    objective = cp.Minimize(port_risk) 
    constraints = [w >= 0, port_return == 1]
    
    prob = cp.Problem(objective, constraints)
    prob.solve()            
    # print(prob.solver_stats.solver_name)
    
    if w.value is not None:
        # Normalize weights to sum to 1
        w_normalized = w.value / np.sum(w.value)
        return w_normalized
    else:
        return None

In [ ]:
# List of the solvers CVXPY supports
from cvxpy import installed_solvers
print(installed_solvers())

In [ ]:
# ---- Run Optimizations ----
msr_weights = optimize_max_sharpe(mean_returns, cov_matrix) 
msr_weights

**Minimum Variance Portfolio**

The Minimum Variance Portfolio aims to find asset weights that minimize overall portfolio risk, regardless of expected returns. In this formulation, we minimize the portfolio's variance using the covariance matrix of asset returns. The optimization is subject to two constraints: the weights must sum to 1 (fully invested portfolio), and short-selling is not allowed (weights ≥ 0). This problem is convex and efficiently solved using CVXPY.

In [ ]:
# ---- 2. Minimum Variance Portfolio ----

def optimize_min_variance(cov_matrix):
    
    w = cp.Variable(n)
    
    objective = cp.Minimize(cp.quad_form(w, cov_matrix))
    constraints = [cp.sum(w) == 1, w >= 0]
    
    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    return w.value

In [ ]:
# ---- Run Optimizations ----
mv_weights = optimize_min_variance(cov_matrix) 
mv_weights

**Maximum Return Portfolio**

The Maximum Return Portfolio focuses solely on maximizing expected returns, without considering portfolio risk. Given the vector of mean asset returns, this optimization finds the portfolio weights that yield the highest expected return, subject to two constraints: full investment (weights sum to 1) and no short-selling (weights ≥ 0). This is a linear program and is efficiently solvable using CVXPY.

In [ ]:
# ---- 3. Maximum Return Portfolio ----

def optimize_max_return(mean_returns):
    
    w = cp.Variable(n)
    
    objective = cp.Maximize(mean_returns @ w)
    constraints = [cp.sum(w) == 1, w >= 0]
    
    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    return w.value

In [ ]:
# ---- Run Optimizations ----
mr_weights = optimize_max_return(mean_returns)
mr_weights

**Portfolio Composition after Optimization**

In [ ]:
# MV Weights
mvwtdf = pd.DataFrame(100*mv_weights, index=assets, columns=['wts'])
mvwtdf.iplot(kind='pie', showlegend=True, title='MV Weights')

## Efficient Frontier

The Efficient Frontier is formed by a set of portfolios offering the highest expected portfolio return for a certain volatility or offering the lowest volatility for a certain level of expected returns. 

**`Minimize Portfolio Risk for a Target Return`:** 

* Risk objective and Return constraint

$$\underset{w_1,w_2,\dots,w_n}{minimize} \space\space \sigma^2_{p}(w_1,w_2,\dots,w_n)$$

subject to,

$$E[R_p] = m$$


**`Maximize Portfolio Return for a Target Risk`**:
* Return objective and Risk constraint

$$\underset{w_1,w_2,\dots,w_n}{maximize} \space\space E[R_p(w_1,w_2,\dots,w_n)]$$

subject to,

$$\sigma^2_{p}(w_1,w_2,\dots,w_n)=v^2$$

where, $\sum_{i=1}^{n}w_i=1$, 

$m$ is the target return, and 

$v$ is the target volatility for the above objectives. 

We can use numerical optimization techniques such as quadratic programming to solve these problems. The goal is to find the optimal portfolio weights that minimize or maximize the objective function, while satisfying the specified constraints. These techniques allow us to compute the full efficient frontier by iterating across a range of return or risk levels.

In [ ]:
# ---- 4. Efficient Frontier ----

def efficient_frontier(mean_returns, cov_matrix, points=100):
    
    target_returns = np.linspace(mean_returns.min(), mean_returns.max(), points)
    frontier = []

    for target in target_returns:
        
        w = cp.Variable(n)
        port_risk = cp.quad_form(w, cov_matrix)
        
        objective = cp.Minimize(port_risk)
        constraints = [cp.sum(w) == 1, w >= 0, mean_returns @ w == target]
         
        prob = cp.Problem(objective, constraints)
        prob.solve()
        
        if w.value is not None:
            vol = np.sqrt(w.value.T @ cov_matrix @ w.value)
            frontier.append((vol, target))
            
    return np.array(frontier)

**Plot Efficient Frontier**

In [ ]:
# ---- Get Optimized Portfolio Statistics ----
def get_stats(w):
    ret = mean_returns @ w
    vol = np.sqrt(w.T @ cov_matrix @ w)
    return 100*ret, 100*vol

msr_ret,msr_vol = get_stats(msr_weights)
mv_ret, mv_vol = get_stats(mv_weights)
mr_ret, mr_vol = get_stats(mr_weights)

In [ ]:
# ---- 5. Plot Efficient Frontier ----

ef_curve = efficient_frontier(mean_returns, cov_matrix)
ef_port = 100 * pd.DataFrame(ef_curve, columns=['Volatility', 'Return'])

fig = ef_port.iplot(
    kind='scatter',
    x='Volatility', 
    y='Return', 
    title='Efficient Frontier Portfolio', 
    name='Efficient Frontier', 
    xaxis_title="Volatility (Risk)", 
    yaxis_title="Expected Return", 
    showlegend=True
    ) 

fig.add_trace(go.Scatter(x=[msr_vol], y=[msr_ret], 
    marker=dict(size=10, color='green'), text=["Max Sharpe"], name='Max Sharpe'))

fig.add_trace(go.Scatter(x=[mv_vol], y=[mv_ret], 
        marker=dict(size=10, color='blue'), text=["Min Variance"], name='Min Variance'))    

---
[Kannan Singaravelu](https://www.linkedin.com/in/kannansi) | Refer [Quantmod](https://kannansingaravelu.com/quantmod/), [Plotly](https://plotly.com/python/) and [CVXPY](https://www.cvxpy.org/index.html) for more information.